In [1]:
# Package code into MAP
!monai-deploy package -l DEBUG -b nvcr.io/nvidia/pytorch:22.08-py3 ./prostate_mri_lesion_seg_app --tag lesion_seg_workflow_app:latest -m prostate_mri_lesion_seg_app/models/

[2023-07-25 10:53:42,712] [DEBUG] (app.AIProstateLesionSegApp) - Begin compose
[2023-07-25 10:53:42,714] [DEBUG] (app.AIProstateLesionSegApp) - End compose
[2023-07-25 10:53:44,300] [DEBUG] (app_packager) - FROM nvcr.io/nvidia/pytorch:22.08-py3

    LABEL base="nvcr.io/nvidia/pytorch:22.08-py3"
    LABEL tag="lesion_seg_workflow_app:latest"
    LABEL version="0.5.1"
    LABEL sdk_version="0.5.1"

    ENV DEBIAN_FRONTEND=noninteractive
    ENV TERM=xterm-256color
    ENV MONAI_INPUTPATH=/var/monai/input
    ENV MONAI_OUTPUTPATH=/var/monai/output
    ENV MONAI_WORKDIR=/var/monai/
    ENV MONAI_APPLICATION=/opt/monai/app
    ENV MONAI_TIMEOUT=0
    ENV MONAI_MODELPATH=/opt/monai/models

    RUN apt update \
     && apt upgrade -y --no-install-recommends \
     && apt install -y --no-install-recommends \
        curl \
        unzip \
     && apt autoremove -y \
     && rm -rf /var/lib/apt/lists/*
    
    USER root

    RUN pip install --no-cache-dir --upgrade setuptools==59.5.0 pip==22.3

In [2]:
# Run MAP
!monai-deploy run -l DEBUG lesion_seg_workflow_app:latest /media/jtetreault/Data/Datasets/ProstateX/PROSTATEx/ProstateX-0004 output

Checking dependencies...
--> Verifying if "docker" is installed...

--> Verifying if "lesion_seg_workflow_app:latest" is available...

Checking for MAP "lesion_seg_workflow_app:latest" locally
"lesion_seg_workflow_app:latest" found.

Reading MONAI App Package manifest...
-------------------application manifest-------------------
{
    "api-version": "0.1.0",
    "command": "python3 -u /opt/monai/app/app.py",
    "environment": {},
    "input": {
        "formats": [],
        "path": "input"
    },
    "output": {
        "format": {},
        "path": "output"
    },
    "sdk-version": "0.5.1",
    "timeout": 0,
    "version": "0.5.1",
    "working-directory": "/var/monai/"
}
----------------------------------------------

-------------------package manifest-------------------
{
    "api-version": "0.1.0",
    "application-root": "/var/monai/",
    "models": [
        {
            "name": "organ-5b6bbe150488ed6e34ac61d113aeb4786ec1c4cda9bd18304f6a8a891e93e79d",
            "path": "/o